In [1]:
import csv

import pandas as pd
import sklearn_crfsuite
import os
import numpy as np

In [2]:
# !python -m spacy download en_core_web_md

In [3]:
import spacy
nlp = spacy.load('en_core_web_md')

In [4]:

def strip_iob(iob_tag):
    tag = iob_tag.replace("B-", "")
    tag = tag.replace("I-", "")
    return tag


In [5]:
def is_new_tag(prev, current):
    if "O" in prev:
        prev_t, prev_w = "O", "O"
    else:
        prev_t, prev_w = prev.split("-")
    if "O" in current:
        current_t, current_w = "O", "O"
    else:
        current_t, current_w = current.split("-")

    if prev_w != current_w:
        return True
    else:
        if prev_t =="B" and current_t =="I":
            return False
        elif prev_t =="I" and current_t =="B":
            return True
        elif prev_t =="I" and current_t =="I":
            return False
        else:
            return False

In [6]:
full_path = os.getcwd() #environ['DEMO_DATA_PATH']
full_path

'/mnt1/gaurish/hyperion/vitaFlow/vitaflow/playground/clientx'

In [7]:
def returnSentences(folder="test"):
    sentences = []
    for i in os.listdir(os.path.join(full_path, folder)):
        lemma = []
        pos = []
        ner = []
        shape =[]
        alpha =[]
        stop =[]
        df = pd.read_csv(os.path.join(full_path, folder,i),sep="\t",quoting=csv.QUOTE_NONE)
        df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
        for doc in nlp.pipe(df['0'].astype('unicode').values, batch_size=350,n_threads=-1):
            if doc.is_parsed:
                lemma.append([n.lemma_ for n in doc])
                pos.append([n.pos_ for n in doc])
                ner.append([n.tag_ for n in doc])
                shape.append([n.shape_ for n in doc])
                alpha.append([n.is_alpha for n in doc])
                stop.append([n.is_stop for n in doc])

            else:
                # We want to make sure that the lists of parsed results have the
                # same number of entries of the original Dataframe, so add some blanks in case the parse fails
                tokens.append(None)
                lemma.append(None)
                pos.append(None)
                ner.append(None)
                shape.append(None)
                alpha.append(None)
                stop.append(None)
#         print(pos)        
        df['lemma'] = lemma
        df['pos'] = pos
        df['ner'] = ner
        df['shape'] = shape
        df['alpha'] = alpha
        df['stop'] = stop

        data =  list(zip(*[df[c].values.tolist() for c in ['0','pos','lemma','ner','shape','alpha','stop']]))
        sentences.append(data)
        
    return sentences

In [8]:
#Basically in the test folder we have to keep all the csv files with two rows
test_sents = returnSentences("tobepredicted")

In [9]:
len(test_sents)

3640

In [10]:
prediction_folder =full_path+"/clientx_dataset/clientx_data_iterator/predictions"

In [11]:
!rm -rf clientx_dataset/clientx_data_iterator/predictions
!mkdir -p clientx_dataset/clientx_data_iterator/predictions

In [12]:
def check_n_makedirs(path):
    if not os.path.exists(path):
        print_info("Creating folder: {}".format(path))
        os.makedirs(path)

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)
        
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1][0]
    lemma = sent[i][2][0]
    ner = sent[i][3][0]
    shape = sent[i][4][0]
    alpha = sent[i][5][0]
    stop = sent[i][6][0]
    
    features = {
        'bias': 1.0,
#         'word.lower()': word.lower() if type(word) is str else False,
#         'word[-2:]': word[-2:] if type(word) is str else word,
#         'word[:2]': word[-2:] if type(word) is str else word,
        'word[-4:]': word[-4:] if type(word) is str else word,
        'word[:4]': word[-4:] if type(word) is str else word,
        'word.isupper()': word.isupper() if type(word) is str else False ,
        'word.isalnum()':word.isalnum() if type(word) is str else False ,
        'word.istitle()': word.istitle() if type(word) is str else False,
        'word.isdigit()': word.isdigit() if type(word) is str else False,
#         'word.ishash': "#" in word if type(word) is str else False,
        'word.hasnumber()':hasNumbers(word) if type(word) is str else False,
        'word.isdot': "." in  word if type(word) is str else False,
        'word.uppercharcount': sum(1 for c in word if c.isupper()) if type(word) is str else 0,
        'word.lowercharcount': sum(1 for c in word if c.islower()) if type(word) is str else 0,
        'word.digitcount':sum(1 for c in word if c.isdigit()) if type(word) is str else 0,
        'word.length':len(word) if type(word) is str else 0,
        'word.vocablength':len(set(word)) if type(word) is str else -1,
#         'word.isroad': "road" in word.lower() if type(word) is str else False,
#         'word.hasslash': "/" in word if type(word) is str else False,
#         'word.hashyphen': ":" in word if type(word) is str else False,
#         'word.currency':word in ["pound","$","£","dollar"] if type(word) is str else False,
        'lemma':lemma,
        'postag': postag,
        'postag[:2]': postag[:2],
        'nertag': ner,
        'shape':shape,
        'alpha':alpha,
        'stop':stop,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1][0]
        nertag1 = sent[i-1][2][0]
        lemma1 = sent[i-1][2][0]
        ner1 = sent[i-1][3][0]
        shape1 = sent[i-1][4][0]
        alpha1 = sent[i-1][5][0]
        stop1 = sent[i-1][6][0]
        features.update({
#             '-1:word[-2:]': word1[-2:] if type(word) is str else word,
#             '-1:word[:2]': word1[-2:] if type(word) is str else word,
#             '-1:word.lower()': word1.lower() if type(word1) is str else False,
            '-1:word.istitle()': word1.istitle() if type(word1) is str else False,
            '-1:word.isupper()': word1.isupper() if type(word1) is str else False,
            '-1:word.isdigit()': word1.isdigit() if type(word1) is str else False,
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:lemma':lemma1,
            '-1:nertag': nertag1,
            '-1:shape':shape1,
            '-1:alpha':alpha1,
            '-1:stop':stop1
            
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1][0]
        nertag1 = sent[i+1][2][0]
        lemma1 = sent[i+1][2][0]
        ner1 = sent[i+1][3][0]
        shape1 = sent[i+1][4][0]
        alpha1 = sent[i+1][5][0]
        stop1 = sent[i+1][6][0]
        
        features.update({
#             '+1:word.lower()': word1.lower() if type(word1) is str else False,
            '+1:word.istitle()': word1.istitle() if type(word1) is str else False,
            '+1:word.isupper()': word1.isupper() if type(word1) is str else False,
            '+1:word.isdigit()': word1.isdigit() if type(word1) is str else False,
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:lemma':lemma1,
            '+1:nertag': nertag1,
            '+1:shape':shape1,
            '+1:alpha':alpha1,
            '+1:stop':stop1,
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token,pos,lemma,ner,shape,alpha,stop, label in sent]

def sent2tokens(sent):
    return [token for token,pos,lemma,ner,shape,alpha,stop, label in sent]



In [13]:
X_test = [sent2features(s) for s in test_sents]
# y_test = [sent2labels(s) for s in test_sents]

In [14]:
from sklearn.externals import joblib 
# Load the model from the file 
crf = joblib.load('CRFModelExpt2.pkl')


In [15]:
# Use the loaded model to make predictions 
predictions = crf.predict(X_test)

In [16]:
!rm -rf clientx_dataset/clientx_data_iterator/predictions
!mkdir -p clientx_dataset/clientx_data_iterator/predictions

In [23]:
OUT_DIR ="clientx_dataset/clientx_data_iterator"
_prediction_col="predictions"
from_pred_folder = "tobepredicted"
# Get the files from test folder and zip it with predictions
for each_prediction, file in zip(predictions, os.listdir(os.path.join(full_path,from_pred_folder))):
    print(file)
    df = pd.read_csv(os.path.join(full_path,from_pred_folder,file), sep="\t", quoting=csv.QUOTE_NONE)
    df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

    df[_prediction_col] = [j for i, j in
                          zip(df["0"].astype(str).values.tolist(), each_prediction)]
    
    out_dir = os.path.join(OUT_DIR, "predictions")
    check_n_makedirs(out_dir)
    df.to_csv(os.path.join(out_dir, os.path.basename(file)), index=False)

S02213-20190418-1120-1.csv
S01276-20190514-0929-4.csv
S01248-20190416-1441-20.csv
S02594-20190515-1637-1.csv
S00468-20190425-1458-9.csv
S00995-20190514-1428-1.csv
S03046-20190510-1114-15.csv
S00985-20190515-1429-131.csv
S01075-20190507-1534-1.csv
S02938-20190430-0929-1.csv
S00985-20190515-1429-129.csv
S00985-20190515-1429-41.csv
S00316-20190513-1553-1.csv
S00174-20190502-0939-1.csv
S01294-20190513-1255-1.csv
S00985-20190516-1033-1.csv
S00094-20190425-1617-2.csv
S00985-20190515-1429-181.csv
S00918-20190513-1141-1.csv
S03019-20190508-1143-1.csv
S00660-20190513-1324-1.csv
S01248-20190416-1441-60.csv
S00296-20190416-1631-3.csv
S00985-20190516-1431-2.csv
S00569-20190430-1021-1.csv
S00985-20190515-1423-13.csv
S02396-20190501-1617-4.csv
S01211-20190516-1202-2.csv
doc23201520190509100428-1.csv
S00700-20190509-0937-1.csv
S01248-20190416-1441-8.csv
S01248-20190416-1441-30.csv
S02603-20190508-1625-4.csv
S00468-20190417-1626-1.csv
S00455-20190507-1408-1.csv
S00094-20190510-1412-1.csv
S01079-201905

S02899-20190425-0933-1.csv
S03072-20190513-1200-1.csv
S02277-20190426-0807-1.csv
S03015-20190510-1556-1.csv
S02698-20190507-1552-18.csv
S02594-20190515-1158-1.csv
S01248-20190416-1441-18.csv
S00985-20190515-1429-17.csv
S00985-20190514-1503-1.csv
S01079-20190502-1046-1.csv
S01075-20190507-1534-3.csv
S00881-20190425-1437-2.csv
S02662-20190416-1457-3.csv
S01171-20190515-1127-73.csv
S00094-20190424-1527-2.csv
S00985-20190515-1429-119.csv
S00356-20190502-1623-6.csv
S01143-20190516-1405-2.csv
S00399-20190429-1712-1.csv
S01171-20190515-1127-75.csv
S00859-20190425-1608-1.csv
S01735-20190517-1155-1.csv
S02058-20190515-1240-1.csv
S00455-20190317-0954-2.csv
S00985-20190515-1429-147.csv
S02150-20190510-1032-1.csv
S02608-20190418-0916-1.csv
S00849-20190425-1551-9.csv
S00985-20190514-1210-1.csv
S00413-20190520-1039-1.csv
S00226-20190510-1411-1.csv
S00601-20190502-1030-1.csv
S02960-20190430-1532-1.csv
S00985-20190430-1015-1.csv
S03046-20190510-1114-17.csv
S02869-20190510-1327-2.csv
S02380-20190416-15

S01248-20190416-1441-15.csv
S02985-20190514-1602-1.csv
S02813-20190514-1618-1.csv
Back-Up-Hyperion-Service--09.05.19-1.csv
S00832-20190513-1454-1.csv
S00985-20190514-0935-1.csv
S00356-20190430-0939-1.csv
S00486-20190516-1531-3.csv
S01778-20190516-1625-1.csv
S00543-20190417-1630-1.csv
S01337-20190425-1447-5.csv
S00853-20190502-0932-1.csv
S00531-20190513-1630-2.csv
S02710-20190423-1255-1.csv
S00985-20190514-1426-1.csv
S00360-20190510-0912-2.csv
S00985-20190514-1529-3.csv
S00978-20190515-1058-1.csv
S02608-20190418-0925-1.csv
S00985-20190515-1429-83.csv
S02906-20190425-1128-2.csv
S00805-20190430-1450-6.csv
S01171-20190515-1127-119.csv
S02603-20190501-1105-3.csv
S01248-20190416-1441-3.csv
S00985-20190514-1147-1.csv
S00383-20190510-0957-1.csv
S00876-20190423-1136-1.csv
S03052-20190510-1553-1.csv
S01716-20190502-1144-1.csv
S00739-20190510-1312-5.csv
S02654-20190516-1048-1.csv
S01137-20190509-0904-2.csv
S00935-20190502-1546-10.csv
S01171-20190515-1127-14.csv
S01616-20190515-1255-1.csv
S02983-2

S03086-20190514-1230-1.csv
S00767-20190501-1625-1.csv
S00985-20190513-1425-10.csv
S02489-20190510-1000-1.csv
S00558-20190417-1108-1.csv
S03077-20190514-1333-1.csv
S00406-20190416-1604-1.csv
S01731-20190509-1012-3.csv
S02453-20190502-1438-2.csv
S02596-20190517-1628-1.csv
S01171-20190515-1127-26.csv
S00436-20190516-1543-2.csv
S00558-20190503-0847-1.csv
S00985-20190515-1429-57.csv
S00985-20190515-1423-12.csv
S02825-20190426-1631-1.csv
S00985-20190514-1411-1.csv
S00985-20190515-1429-110.csv
S03064-20190513-0753-1.csv
S01198-20190424-1359-1.csv
S01198-20190424-1412-1.csv
S03027-20190513-1250-1.csv
S00608-20190424-1441-2.csv
S00408-20190517-1408-3.csv
S00985-20190430-0952-1.csv
S02654-20190502-0922-1.csv
S00985-20190515-1429-51.csv
S01303-20190425-1232-1.csv
S00232-20190513-1408-1.csv
S02594-20190515-1643-1.csv
S00685-20190425-0904-2.csv
S00719-20190425-1005-3.csv
INV300005590-Psyon-Havensrock-1.csv
S00996-20190503-1531-1.csv
1266_001-1.csv
S00963-20190509-1302-2.csv
S00464-20190429-0923-2.c

S00690-20190507-0927-1.csv
S01171-20190515-1127-15.csv
S01171-20190515-1127-105.csv
S00174-20190502-0943-1.csv
S02608-20190417-1642-1.csv
S00455-20190507-1626-4.csv
S02770-20190510-1139-1.csv
S01171-20190515-1127-141.csv
S01211-20190516-1157-1.csv
S02107-20190417-1526-2.csv
S00094-20190514-1224-1.csv
S00805-20190502-0913-1.csv
S01171-20190515-1127-123.csv
S00094-20190425-1525-2.csv
S00406-20190417-1402-1.csv
S01143-20190424-1633-2.csv
S01749-20190517-1202-2.csv
S02623-20190509-1559-2.csv
RG153934-1.csv
S00985-20190409-1042-(2)-1.csv
S02906-20190425-1128-1.csv
S01143-20190516-1416-2.csv
S02946-20190509-1604-2.csv
S00711-20190502-0913-1.csv
S02377-20190417-1615-1.csv
S02544-20190517-1053-2.csv
S01379-20190508-1642-1.csv
S02654-20190503-1020-1.csv
S01223-20190425-0850-5.csv
S00094-20190514-1236-1.csv
S02783-20190430-1416-1.csv
S01248-20190416-1441-27.csv
S00918-20190513-1143-1.csv
S02870-20190416-1452-1.csv
S02945-20190430-0952-2.csv
S02004-20190509-1734-2.csv
S00808-20190424-1522-1.csv
S

...

S00849-20190425-1551-20.csv
S00985-20190513-1425-20.csv
S00094-20190515-1427-2.csv
S01171-20190515-1127-13.csv
S03094-20190516-0842-1.csv
S01171-20190515-1127-80.csv
S00010-20190513-1239-2.csv
S02594-20190515-1654-1.csv
S00985-20190513-1645-1.csv
S02484-20190416-1050-1.csv
S01195-20190508-1605-3.csv
S00805-20190502-0913-12.csv
S00985-20190515-1429-112.csv
S01209-20190508-1543-1.csv
S02886-20190426-1128-1.csv
S00985-20190513-1425-8.csv
S00985-20190515-1429-31.csv
S00985-20190513-1418-1.csv
S02794-20190507-1601-1.csv
S02478-20190502-1004-4.csv
S03046-20190510-1114-18.csv
S02677-20190514-1128-1.csv
S02323-20190429-1450-1.csv
S00985-20190515-1429-16.csv
S00985-20190515-1603-1.csv
S01171-20190515-1127-46.csv
S00985-20190515-1429-179.csv
S00455-20190507-1626-5.csv
S00538-20190503-1023-1.csv
S01648-20190501-1439-1.csv
S02940-20190520-1239-2.csv
S02592-20190502-0949-2.csv
S00363-20190426-1126-1.csv
S00985-20190509-1637-1.csv
S01333-20190508-1709-3.csv
S01067-20190426-1123-1.csv
S01021-20190405

S00094-20190515-1423-1.csv
S03016-20190508-0826-1.csv
S00985-20190515-1429-165.csv
S02865-20190416-1326-1.csv
S00795-20190516-1611-1.csv
S00508-20190510-1528-3.csv
S03018-20190508-1112-1.csv
S00985-20190515-1429-183.csv
S02354-20190507-1055-1.csv
S03094-20190516-0842-2.csv
S01061-20190508-1553-2.csv
S00094-20190424-1530-2.csv
S02964-20190513-0938-3.csv
S03018-20190508-1112-3.csv
S00558-20190516-0946-1.csv
S00406-20190517-1455-1.csv
S03069-20190513-1134-2.csv
S00094-20190508-1216-1.csv
S01171-20190515-1127-124.csv
S00356-20190502-1618-9.csv
S00300-20190418-1049-3.csv
S00094-20190429-1553-1.csv
S02883-20190418-1415-2.csv
S00455-20190509-1026-1.csv
S00935-20190502-1546-3.csv
S00918-20190513-1143-2.csv
S01171-20190515-1127-101.csv
S01143-20190513-1012-4.csv
S02673-20190426-1435-10.csv
S00316-20190513-1554-2.csv
S00985-20190515-1429-59.csv
S00589-20190507-1128-1.csv
S02928-20190426-1450-1.csv
S00985-20190513-1523-1.csv
S01171-20190515-1127-66.csv
S02763-20190425-1126-2.csv
S00880-20190430-1

1392613-2.csv
S00805-20190430-1450-3.csv
S03028-20190509-1140-1.csv
S00547-20190426-0950-1.csv
S00094-20190425-1636-1.csv
S00675-20190502-1714-1.csv
S00859-20190430-1104-1.csv
S00054-20190502-1534-1.csv
S00918-20190517-1415-2.csv
S00300-20190418-1058-1.csv
S02760-20190416-1540-1.csv
S02935-20190430-1218-1.csv
S00985-20190515-1429-3.csv
S00022-20190430-0938-1.csv
S00985-20190515-1227-1.csv
S01248-20190416-1441-7.csv
S02834-20190517-1618-1.csv
S02453-20190502-1438-1.csv
S01944-20190502-1559-1.csv
S01868-20190426-1004-1.csv
S00849-20190425-1551-8.csv
S00985-20190515-1429-167.csv
mohammed_1.jakariya_08-05-2019_14-37-56-2.csv
S02908-20190425-1457-1.csv
S01248-20190416-1441-112.csv
S01097-20190425-1533-1.csv
S02594-20190515-1640-1.csv
S01171-20190515-1127-45.csv
S00985-20190513-1425-13.csv
S00399-20190515-1721-2.csv
S00406-20190510-1539-12.csv
S00409-20190513-1458-1.csv
S00455-20190507-1626-3.csv
S01276-20190509-1620-4.csv
S00300-20190418-1101-1.csv
S00837-20190516-1547-1.csv
S01097-20190425

S03048-20190510-1456-2.csv
S01143-20190513-1012-2.csv
S01248-20190416-1441-55.csv
S00985-20190430-1009-1.csv
S00208-20190503-0942-2.csv
S00985-20190514-1207-1.csv
S02695-20190429-1224-2.csv
1692582-1.csv
S00837-20190516-1547-9.csv
S00849-20190425-1551-7.csv
S01344-20190509-1018-1.csv
S00396-20190513-1143-1.csv
S03070-20190513-1524-2.csv
S00601-20190502-0953-1.csv
S02267-20190516-1409-1.csv
S00120-20190515-1554-6.csv
S02909-20190502-1637-1.csv
S02881-20190418-1525-1.csv
S02951-20190509-1114-1.csv
S02818-20190514-1549-1.csv
S00094-20190424-1533-1.csv
S02489-20190513-1452-1.csv
S00232-20190513-1418-1.csv
S00371-20190417-1037-3.csv
S03066-20190513-1015-1.csv
S00985-20190515-1429-115.csv
S00985-20190516-1431-4.csv
S01483-201900507-0913-1.csv
S00985-20190515-1134-1.csv
S02698-20190507-1552-3.csv
S02247-20190514-1015-2.csv
S02201-20190507-1154-1.csv
S00985-20190515-1429-197.csv
S01683-20190520-1327-2.csv
S02677-20190430-1405-1.csv
S00120-20190515-1554-2.csv
S00675-20190520-1058-2.csv
S00022-2

S00399-20190515-1617-2.csv
S00849-20190425-1551-16.csv
S00985-20190515-1429-20.csv
S02641-20190503-1502-1.csv
S00057-20190426-1539-1.csv
S00487-20190417-1507-1.csv
S01171-20190515-1127-39.csv
S00184-20190508-1557-1.csv
S00056-20190515-1016-1.csv
S00985-20910430-1029-1.csv
S02594-20190516-0908-1.csv
S00985-20190515-1423-1.csv
S00985-20190515-1429-196.csv
S01248-20190416-1441-26.csv
S00880-20190430-1605-1.csv
S00837-20190516-1547-3.csv
S01567-20190513-1550-2.csv
S00775-20190416-1237-4.csv
S02991-20190503-1543-1.csv
S00880-20190502-1008-1.csv
S02544-20190424-1436-1.csv
S03108-20190516-1627-1.csv
1292_001-1.csv
SKM_454e19050812470_0010-1.csv
S01303-20190515-1211-3.csv
S01056-20190426-1628-1.csv
S01143-20190513-1012-5.csv
S00040-20190502-1054-1.csv
S01171-20190515-1127-36.csv
S02462-20190520-1026-1.csv
S01137-20190510-1136-1.csv
S00120-20190426-1627-3.csv
S02993-20190508-1254-1.csv
S00114-20190503-1721-1.csv
S01885-20190502-1017-1.csv
S02608-20190417-1618-1.csv
S02714-20190417-1450-1.csv
Wa

In [26]:
len(predictions[-1]),df.shape

(358, (358, 3))

In [28]:
import random
SEED = 448

file_list = os.listdir(out_dir)
random.seed(SEED)
random.shuffle(file_list)

print(file_list[:30])

['S00538-20190417-0938-3.csv', 'S01097-20190429-1253-1.csv', 'S02999-20190503-1552-1.csv', 'S00396-20190513-1143-2.csv', 'S02309-20190430-1531-1.csv', 'S03047-20190510-1356-1.csv', 'S02673-20190426-1435-8.csv', 'S00985-20190515-1429-193.csv', 'S00573-20190508-0914-2.csv', 'S00985-20190514-0939-1.csv', 'S00455-20190509-1603-5.csv', 'S02919-20190425-1234-4.csv', 'S01152-20190508-1730-1.csv', 'S02087-20190503-1712-1.csv', 'S00985-20190515-1630-1.csv', 'S00985-20190513-1425-23.csv', 'S00985-20190515-1429-52.csv', 'S03095-20190515-1450-1.csv', 'S00532-20190514-1523-1.csv', 'S02900-20190425-1010-1.csv', 'S01097-20190429-1253-2.csv', 'S00985-20190515-1429-67.csv', 'S02304-20190423-1555-1.csv', 'S03039-20190509-1621-1.csv', 'S00094-20190425-1636-2.csv', 'S01143-20190516-1505-2.csv', 'S02511-20190502-1620-2.csv', 'S00935-20190502-1541-1.csv', 'S01143-20190510-1023-4.csv', 'S00985-20190514-1458-1.csv']


In [29]:
! less {out_dir}/{file_list[0]}

0,1,predictions
Howden,O,O
Insurance,O,O
Brokers,O,O
Ltd,O,O
(,O,O
PLS,O,O
ask,O,O
for,O,O
password,O,O
',O,O
LILLY,O,O
',O,O
-,O,O
(,O,O
ID,O,O
:,O,O
2768,O,O
-,O,O
RT,O,O
:,O,O
043946,O,O
),O,O
:entx_dataset/clientx_data_iterator/predictions/S00538-20190417-0938-3.csv

In [33]:
! rm -rf tobeanalysed
! mkdir -p tobeanalysed

In [35]:
import shutil
for i in file_list[:30]:
    print(os.path.basename(i))
    shutil.copy(os.path.join(out_dir,i), os.path.join('./tobeanalysed/',os.path.basename(i)))

S00538-20190417-0938-3.csv
S01097-20190429-1253-1.csv
S02999-20190503-1552-1.csv
S00396-20190513-1143-2.csv
S02309-20190430-1531-1.csv
S03047-20190510-1356-1.csv
S02673-20190426-1435-8.csv
S00985-20190515-1429-193.csv
S00573-20190508-0914-2.csv
S00985-20190514-0939-1.csv
S00455-20190509-1603-5.csv
S02919-20190425-1234-4.csv
S01152-20190508-1730-1.csv
S02087-20190503-1712-1.csv
S00985-20190515-1630-1.csv
S00985-20190513-1425-23.csv
S00985-20190515-1429-52.csv
S03095-20190515-1450-1.csv
S00532-20190514-1523-1.csv
S02900-20190425-1010-1.csv
S01097-20190429-1253-2.csv
S00985-20190515-1429-67.csv
S02304-20190423-1555-1.csv
S03039-20190509-1621-1.csv
S00094-20190425-1636-2.csv
S01143-20190516-1505-2.csv
S02511-20190502-1620-2.csv
S00935-20190502-1541-1.csv
S01143-20190510-1023-4.csv
S00985-20190514-1458-1.csv
